## Creating the Bicrystal
In this code the bicrystal is created and saved as a lammps-dump file.
This tutorial has the following sections:
1. Load relevant modules
2. define necessary parameters
    + Lattice (here Mg)
    + Cut-off radius (r_cut)
3. uploading necessary files
    + `pkl_name_prop`: A dictionary which contains the bicrystallography properties for each Σ-miorientation.
    + `pkl_name_inds`: A dictionary which contains the list of boundary-plane orientations in hkl_sig_inds.
4. 

In [1]:
import byxtal.tools as gbt
import byxtal.lattice as gbl
import pickle as pkl
import numpy as np
import numpy.linalg as nla
import byxtal.misorient_fz as mfz
import gbpy.generate_hkl_indices as ghi
import gbpy.util_funcs_create_byxtal as uf

1. `l1`         : Lattice object
2. `lat_par`    : Lattice parameter
3. `r_cut`      : Cutoff radius
4. `A_cut`      : Cutoff Area
5. `l_p_po`   : $\Lambda_{p}^{po}$
6. `l_po_p`   : $\Lambda_{po}^{p}$

In [2]:
########################################################################
l1=gbl.Lattice('Mg_sun')
lat_par = l1.lat_params['a']
rCut = lat_par*3
A_cut = (rCut+lat_par)**2

l_p_po = l1.l_p_po
l_po_p = nla.inv(l_p_po)

`pkl_name_prop`: The calculated bicrystallography properties for each $\Sigma$-miorientation includes:

1. `sig_mats`: The $\Sigma$-miorientation in the primitive $p$ reference frame. For example, we use the variable `T_p1top2_p1`
2. `csl_mats`: The CSL lattice basis vectors in the primitive $p$ reference frame. For example, we use the variable `l_csl_p` denoting $\Lambda_{csl}^p$
3. `dsc_mats`: The DSC lattice basis vectors in the primitive $p$ reference frame. For example, we use the variable `l_dsc_p` denoting $\Lambda_{dsc}^p$
4. `bxt_symm_props`: The symmetry aspects of the $\Sigma$-miorientation
    + `bxt_symm_grp`: The point-group symmetry of the bicrystal. This is the symmetry for the Boundary-plane orientations.
    + `symm_grp_ax`: The axes of the symmetry point-group.

`pkl_name_ind`: The bicrystallography properties for each Σ-miorientation includes:
1. `l_p_po1`: $\Lambda_p^{po}$
2. `miller_inds`: Indices of boundary-plane orientation for a given Σ-misorientation

In [3]:
pkl_name_prop = l1.elem_type+'_byxtal_props.pkl'
jar = open(pkl_name_prop, 'rb');

csl_props = pkl.load(jar)
sig_mats = csl_props['sig_mats']
csl_mats = csl_props['csl_mats']
dsc_mats = csl_props['dsc_mats']
bxt_symm_props = csl_props['csl_symm_props']
jar.close()

pkl_name_ind = 'bp_list_' + l1.elem_type + '.pkl'
jar = open(pkl_name_ind, 'rb');

bp_list = pkl.load(jar)
l_p_po = bp_list['l_p_po']
hkl_sig_inds = bp_list['miller_inds']
jar.close()


In [4]:
s1_keys = list(hkl_sig_inds.keys())
ind1 = 1
sig_id = s1_keys[ind1]
print(sig_id)
hkl_inds = hkl_sig_inds[sig_id]
l_csl_p1 = csl_mats[sig_id]
l_csl_po1 = l_p_po.dot(l_csl_p1)

13_1


In [6]:
for hkl_ct in range(1):
    tct1 = hkl_ct
    hkl1 = np.zeros((1,3), dtype='int64')
    hkl1[0,:] = hkl_inds[tct1,:]
    l_CSLbpb_CSLp = {}
    l_CSLbpbSig_CSLp = {}

    l_CSLbpb_CSLp_mat = ghi.compute_hkl_bpb(hkl1)
    l_CSLbpbSig_CSLp_mat = ghi.gen_Acut_bpb(l_CSLbpb_CSLp_mat, l_csl_po1, rCut, A_cut)

    l_Cbpb_Cp  = l_CSLbpb_CSLp_mat[0]
    l_CRbpb_Cp = l_CSLbpbSig_CSLp_mat[0]

    l_bp_po1 = l_csl_po1.dot(l_CRbpb_Cp)

    symm_grp_ax = bxt_symm_props[sig_id]['symm_grp_ax'];
    bp_symm_grp = bxt_symm_props[sig_id]['bxt_symm_grp'];

    l_p2_p1 = sig_mats[sig_id]
    gb_ID = uf.get_gb_uID(l1, l_bp_po1, l_p2_p1, l_p_po, bp_symm_grp, symm_grp_ax, sig_id)
    print(hkl_ct)
    print(gb_ID)

0
Mg_sun_S13_1_N1_0_0_1_N2_0_0_-1


In [7]:
zCut = 25*l1.lat_params['a']

threeD_upts, sim_cell2 = uf.create_half_cryst(l1, l_csl_p1, l_CRbpb_Cp, l_p_po, 'upper', zCut)

l_p1_p2 = nla.inv(l_p2_p1)
l_csl_p2 = l_p1_p2.dot(l_csl_p1)
threeD_lpts, sim_cell1 = uf.create_half_cryst(l1, l_csl_p2, l_CRbpb_Cp, l_p_po, 'lower', zCut)

In [8]:
gb_attr = {}
gb_attr['cell'] = sim_cell1
gb_attr['upts'] = threeD_upts
gb_attr['lpts'] = threeD_lpts
gb_attr['lattice'] = l1.elem_type

pkl_name = 'gb_attr_' + gb_ID + '.pkl'
jar = open(pkl_name,'wb')
pkl.dump(gb_attr, jar)
jar.close()

Lammps dump file is saved!

In [10]:
import gbpy.lammps_dump_writer as ldw
initial_dump = 'gb_attr_' + gb_ID + '.out'
box_bound, dump_lamp, box_type = ldw.lammps_box(pkl_name)  # lammps creates from the pkl file
ldw.write_lammps_dump(initial_dump, box_bound, dump_lamp, box_type)